# Foreword

With the new awakening system QoL changes, player can now use multiple 0★ gears in 1 attempt of gear awakening. Unsure on whether this could open up new optimize route for awakening, I decided to developed a script in place to do the exploration.

Similar to the old reddit guide: https://www.reddit.com/r/Kings_Raid/comments/7doken/guide_optimal_gear_awakening_strategy/ ,
I use a decision-tree based method to calculate the average cost of awakening gear. I think the old thread did a very good explanation of this decision-tree approach, so I will skip it, you can refer to the hyperlink above for the gist of it. Basically the script just walk down all the possible routes in the decision-tree of awakening gear, and find the route that __on average__ cost us the least amount of gear.

To ease the computation load, I omitted routes where a higher ★ gear is used after lower ★ gear. For example, once you had used a 2★ gear as fodder for awakening, the program will no longer consider taking the route of using even higher ★ gear (>= 3★) for awakening (so no such route as using a 2★, then a 3★, etc, because it doesnt make sense for these routes to be optimal anyway)



# Basic Script Explanation

Very nerdy and unnecessary stuff here just for reference, so those who are interested can know in details how the script works ~~and scold/flame/bash me for writing such an ameteur script with super poor explanation~~.

A node is simply a point in decision-tree, it may be a starting node, or it may had already walk down a few other nodes before it. All the previous route(s) taken by this node will be recorded within it, and it would also had calculated the average cost, as well as the cumulated probability up until this node, given the routes it has taken previously.

A node is represented by a NamedTuple in Python. Which is simply a container that store information based on name:

.history = List, stored number of gear used in all prev attempts of this Node

.fail_bonus = float, failure bonus if continue to awaken from this node

.cum_prob = List, stored the probability of end up success in all the previous route(s) taken by this Node

.cum_avg = float, cumulated average gear used for awakening up until this node (usually an incomplete calculation, only completed when node ended)

.ended = Boolean, whether this node had reached its end point in decision tree


The script first assess all the possible starting point (e.g: from 3★ to 4★, the starting point could be using 1,2,3,4,5,6,7 number of gear (N) for awakening, using 8 gear is not considered a valid choice because you had reached 100% success chance)

Once it knows all the possible starting point, it will initiate the first node in the decision-tree using the largest N available in the list, calculate all the info (cum_prob, cum_avg, etc) for this node, and put this single starting node into a List (nodes_list). 

Then, the program will loop infinitely until the nodes_list is exhausted, and do the following:

1. Take one of the node from the list, remove/exhaust it from the nodes_list 
2. Assess all the different possible route(s) to take next for this node
3. Calculate the next node for every possible route(s) found above.
4. If those newly calculated node(s) is ended (reach 100% success chance due to the piling up of failure bonus, or when it reach 0% increment in failure bonus - which is the infinite decision tree route), their result will be recorded and reported at the end of the script run.
5. Else if, those node(s) still have possible route(s) to go on, they will all be added back to the nodes_list.

(So although nodes_list started with only 1 node, more nodes will subsequently be added to this List as the program walk down the different possible routes and create a node for each of the route undertaken)

The above script will loop until every nodes in nodes_list reached its end. 

Then the next possible starting point will be used to initiate yet another single starting node, and put into the List (nodes_list). Then the program will loop again, until all the routes using these new starting point are also processed as above (i to v).

Then the next possible starting point will be used... and so on... and so on... until all possible starting point was also exhausted. Then the scipt ends here and reported all the results, sorting for the lowest average cost route.


# Difference with Old Guide

There are a few difference to the old thread as well:

Firstly, for 2★ gear and beyond, I believed the old guide only consider the "Actual Cost" of the gear in its decision tree calculation. For example, since it has been found out from its 1★ to 2★ decision-tree that a 2★ gear will need an average of 3.7012 0★ gear to make, the cost of using 2★ gear as fodder to awaken was treated as 3.7012 0★ gear (I will call this "Actual Cost"), not 4 piece of 0★ gear which will be the cost for going 100% success route to make 2★ fodder. (I will call this "Raw Cost").

The thing is... too often I had heard people said that they are NOT going to gamble for awakening a 2 star fodder because the potential gain is too low and the largest possible loss is just too large. As such, in my script, I had attempted the decision-tree calculation both with using the "Actual Cost" and the "Raw Cost", and it actually does return slightly different result for these two cases, at least for the 4★ to 5★ decision-tree where 2★ fodder is actually used.


Next, for case of awakening with 0% increment in failure bonus (infinite route in decision-tree), the old guide used a calculus(?) based approach to get the average cost of taking that branch. Unfortunately, I had returned most of my math knowledge to my teacher since I graduated, and cannot wrap my head around how he arrive with the formula (https://pastebin.com/qKLDXS0M line 135-159). Personally, my intuition is telling me the incremental cost from taking this branch should just be (1/P), not the ((1/P) - 1) in the old guide code. 

Anyway, to be safe in term of math, I am using a more "brute-force" method. Whenever a node in decision tree reached such infinite route, it will continue to walk down the infinite route for 1000 times before it was forced ended and report its average cost result. I can assure you that 1000 time is alot and will confidently give you the true result.


In [1]:
from collections import namedtuple

In [2]:
# A node is used to store information on any point in a decision tree

Node = namedtuple('Node', ['history', 'fail_bonus', 'cum_prob' , 'cum_avg', 'ended'])

In [3]:
# Initiliaze a starting "zero" node

zero_node = Node(history=[],
                fail_bonus=0,
                cum_prob=[],
                cum_avg=0,
                ended=False)

In [4]:
zero_node

Node(history=[], fail_bonus=0, cum_prob=[], cum_avg=0, ended=False)

In [5]:
count_to_star_dict = {1:"0★", 2:"1★", 3:"1★+0★", 4:"2★", 5:"2★+0★", 6:"2★+1★", 7:"2★+1★+0★", 8:"3★", 9:"3★+0★",
                         10:"3★+1★", 11:"3★+1★+0★", 12:"3★+2★", 13:"3★+2★+0★", 14:"3★+2★+1★", 15:"3★+2★+1*+0★", 16:"4★"}

In [6]:
def Round_Down(float_number): # To get the exact failure bonus in KR
    return (int(float_number*100))/100

In [7]:
def Dump_Down_History(LIST):
    
    '''
    Format output result from Node.history list to be more human friendly. Reduce the final spam route with a "+" sign.
    
    Input:
        LIST: list, from Node.history 
        
    Output:
        LIST: list, a more human readable version of Node.hisotry 
    '''
    
    index_list = range(len(LIST)-1)
    
    for i in index_list:
        
        if len(set(LIST[i:])) <= 1: # All item is the same in this slice of list start from index i to the end
            return LIST[:i+1] + ["+"]
        else:
            continue
            
    return LIST # No processing is needed, return original list

In [8]:
def calc_new_node(PREV_NODE, N, USE_ACTUAL_COST):
    
    '''
    
    Calculate the next node in the decision tree. Given the previous node (PREV_NODE), 
    and the number of gear (N) that you are putting in for this round of awakening. 
    
    
    Input:
    
        PREV_NODE = NamedTuple, from the previous node, contains the following name:
            .history = List, stored number of gear used in all prev attempts of this Node
            .fail_bonus = float, failure bonus if continue awakening from this node
            .cum_prob = float, cumulated probability of reaching this node
            .cum_avg = float, cumulated average gear use (incomplete calculation, only complete when Node ended)
            .ended = Boolean, whether this node had reached its end of calculation
            
        N = integer, number of gear to use for this round of awakening
        
        USE_ACTUAL_COST = Boolean. If True, use the "Actual Cost" of ★ gear. Else use the "Raw Cost"
    
    Output:
    
        A new Node after current round of awakening
    
    '''
    
    global chance_dict
    global cost_dict
    
    cur_chance = min( 1, chance_dict[N] + PREV_NODE.fail_bonus) # Success chance for this round of awakening, capped at 100%
    cur_fail_bonus = Round_Down(chance_dict[N]/3) # If this node failed T.T , this is the amount of fail bonus that will be added
    
    ## Two cases for calculating average gear used for this node, one where there is still failure bonus (finite); one where failure bonus = 0
    # Case1
    if cur_fail_bonus >= 0.01: # Fail bonus >= 1% ; finite cases

        # Average gear used for this node: (Probability of reaching this node)(Probability of awakening success)(Cumulated gear used up to and include this node)
        #                                (1 - cumulated_probability)(cur_chance)(cumulated_gear_used)
        
        if USE_ACTUAL_COST: 
            total_used = sum([cost_dict.get(count) for count in PREV_NODE.history]) + cost_dict.get(N)
        else:
            total_used = sum(PREV_NODE.history) + N
            
        cur_prob = [ (1-sum(PREV_NODE.cum_prob)) * (cur_chance) ]
        cur_avg =  cur_prob[0] * (total_used)
        cur_list = [N] # item to be append to the history list
        
        node_ended = (cur_chance >= 1) # True when reached 100% enhance success rate due to prev accumulated failure bonus. Node ended here
                                       # False otherwise
    # Case2    
    else: # Not even 1% of failure bonus, infinite cases calculation. Node also ended here
        
        cur_avg = 0
        cur_prob = []
        cur_list = [] # item to be append to the history list
        
        if USE_ACTUAL_COST:
            total_used = sum([cost_dict.get(count) for count in PREV_NODE.history])
        else:
            total_used = sum(PREV_NODE.history) 

        for x in range(1000): ## Large n to simulate n = infinity, doing infinite decision tree calculation. n=1000 is more than overkill

            total_used += N
            xth_prob = (1-sum(PREV_NODE.cum_prob)) * ((1-cur_chance)**x) * cur_chance
            #          (walk thru finite case      (Walk thru current x-th
            #           decision tree)              node's prob of success)

            xth_avg = total_used * xth_prob
            cur_avg += xth_avg
            
            if x <= 30: # Only append 30 more result to the list to save memory, the real decision tree calc still go to 1000 time
                cur_prob.append(xth_prob)
                cur_list.append(N)
            
        node_ended = True
        
    returned_node = Node( history = PREV_NODE.history + cur_list,
                          fail_bonus = PREV_NODE.fail_bonus + cur_fail_bonus,
                          cum_prob = PREV_NODE.cum_prob + cur_prob,
                          cum_avg = PREV_NODE.cum_avg + cur_avg,
                          ended = node_ended)
        
    return returned_node
    

In [9]:
## real calc to find optimized route for awakening gear, given the possible list of N (Number of gear use for awakening),
#  and the dictionary that pair N with success rate

def Find_Opt_Route(N_LIST, use_actual_cost=False):
    
    '''
    This function create the first node for every possible value of N, add that Node into the nodes_list.
    Afterwards, the function will infinitely take out the last item in nodes_list, used it to calculate
    all the possible next nodes. If those nodes had reached its end point (100% starring success chance /
    0% failure bonus - infinite route), the ended nodes will be append to another list called ended_nodes. Else
    if the nodes is not yet ended, append the newly created nodes into nodes_list.
    
    When all the nodes in nodes_list were ended. The function return the list ended_nodes. The list contained
    all the possible end points in your decision tree.
    
    '''
    
    ended_nodes = []

    for n in N_LIST: ## Different starting point

        # Initiate the first node into the list
        nodes_list = [calc_new_node(zero_node,n,use_actual_cost)]

        while len(nodes_list) >= 1: # As long as there is still nodes not ended

            cur_node = nodes_list[-1] # Always process the last node
            del nodes_list[-1] # Remove this node from list since it had been processed
            
            ## A very special case for spamming only 1 gear at 5star, node ended the moment it is created because it enter infinite case immediately
            if cur_node.ended: continue
                
            n2_list = [x for x in range( 1 , cur_node.history[-1] + 1 )]
            n2_list.reverse() # List of possible route to take (number of gear to use for starring for each route)

            for n2 in n2_list: # Calc the next node for each route

                temp_node = calc_new_node(cur_node,n2,use_actual_cost)

                if temp_node.ended: # This node calculation is done, result can be reported !
                    ended_nodes.append(temp_node)

                else: # Incomplete node, add to nodes_list for further processing
                    nodes_list.append(temp_node)
                    
    return ended_nodes



# 1 STAR TO 2 STAR



In [10]:
## 1star to 2 star

count_list = [1,2]
ActualCost_list = [1,2]
success_chance_list = [0.5,1]

chance_dict = {}
cost_dict = {}

for count,chance in zip(count_list,success_chance_list):
    chance_dict[count] = chance
    
for count,cost in zip(count_list,ActualCost_list):
    cost_dict[count] = cost
    
n_list = [n for n in reversed(range(1,len(count_list)))]

print("n_list: ", n_list)
print("cost_dict: ", cost_dict)
print("chance_dict: ",chance_dict)

n_list:  [1]
cost_dict:  {1: 1, 2: 2}
chance_dict:  {1: 0.5, 2: 1}


In [11]:
## Use "Raw Cost"

all_routes_2star_raw = Find_Opt_Route(n_list, use_actual_cost=False)
all_routes_2star_raw.sort(key=lambda x: x.cum_avg)

for node in all_routes_2star_raw:
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(node.history)))

Avg = 1.7012 ; Route = [1, '+']


In [12]:
## Use "Actual Cost"

all_routes_2star_act = Find_Opt_Route(n_list, use_actual_cost=True)
all_routes_2star_act.sort(key=lambda x: x.cum_avg)

for node in all_routes_2star_act:
    
    templist = [count_to_star_dict.get(item) if count_to_star_dict.get(item) != None else item for item in node.history]
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(templist)))

Avg = 1.7012 ; Route = ['0★', '+']


# 2 STAR TO 3 STAR


In [13]:
## 2star to 3 star

count_list = [1,2,3,4]
ActualCost_list = [1,2,3,3.7012]
success_chance_list = [0.25,0.5,0.75,1]

chance_dict = {}
cost_dict = {}

for count,chance in zip(count_list,success_chance_list):
    chance_dict[count] = chance
    
for count,cost in zip(count_list,ActualCost_list):
    cost_dict[count] = cost
    
n_list = [n for n in reversed(range(1,len(count_list)))]

print("n_list: ", n_list)
print("cost_dict: ", cost_dict)
print("chance_dict: ",chance_dict)

n_list:  [3, 2, 1]
cost_dict:  {1: 1, 2: 2, 3: 3, 4: 3.7012}
chance_dict:  {1: 0.25, 2: 0.5, 3: 0.75, 4: 1}


In [14]:
## Use Raw Cost

all_routes_3star_raw = Find_Opt_Route(n_list)
all_routes_3star_raw.sort(key=lambda x: x.cum_avg)

for node in all_routes_3star_raw:
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(node.history)))

Avg = 2.7954 ; Route = [1, '+']
Avg = 3.0402 ; Route = [2, 1, '+']
Avg = 3.2771 ; Route = [2, 2, 1, '+']
Avg = 3.3806 ; Route = [2, 2, 2, 1, '+']
Avg = 3.4019 ; Route = [2, 2, 2, 2, 1, '+']
Avg = 3.4024 ; Route = [2, '+']
Avg = 3.4509 ; Route = [3, 1, '+']
Avg = 3.5904 ; Route = [3, 2, 1, '+']
Avg = 3.6317 ; Route = [3, 2, 2, 1, '+']
Avg = 3.6362 ; Route = [3, 2, '+']
Avg = 3.7500 ; Route = [3, '+']


In [15]:
# Use Actual cost

all_routes_3star_act = Find_Opt_Route(n_list, use_actual_cost=True)
all_routes_3star_act.sort(key=lambda x: x.cum_avg)

for node in all_routes_3star_act:
    
    templist = [count_to_star_dict.get(item) if count_to_star_dict.get(item) != None else item for item in node.history]
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(templist)))

Avg = 2.7954 ; Route = ['0★', '+']
Avg = 3.0402 ; Route = ['1★', '0★', '+']
Avg = 3.2771 ; Route = ['1★', '1★', '0★', '+']
Avg = 3.3806 ; Route = ['1★', '1★', '1★', '0★', '+']
Avg = 3.4019 ; Route = ['1★', '1★', '1★', '1★', '0★', '+']
Avg = 3.4024 ; Route = ['1★', '+']
Avg = 3.4509 ; Route = ['1★+0★', '0★', '+']
Avg = 3.5904 ; Route = ['1★+0★', '1★', '0★', '+']
Avg = 3.6317 ; Route = ['1★+0★', '1★', '1★', '0★', '+']
Avg = 3.6362 ; Route = ['1★+0★', '1★', '+']
Avg = 3.7500 ; Route = ['1★+0★', '+']


# 3 STAR TO 4 STAR


In [16]:
## 3star to 4star

count_list = [1,2,3,4,5,6,7,8]
ActualCost_list = [1,2,3,3.7012,4.7012,5.7012,6.7012,6.4966]
success_chance_list = [0.1,0.25,0.37,0.5,0.62,0.75,0.87,1]

chance_dict = {}
cost_dict = {}

for count,chance in zip(count_list,success_chance_list):
    chance_dict[count] = chance
    
for count,cost in zip(count_list,ActualCost_list):
    cost_dict[count] = cost
    
n_list = [n for n in reversed(range(1,len(count_list)))]

print("n_list: ", n_list)
print("cost_dict: ", cost_dict)
print("chance_dict: ",chance_dict)

n_list:  [7, 6, 5, 4, 3, 2, 1]
cost_dict:  {1: 1, 2: 2, 3: 3, 4: 3.7012, 5: 4.7012, 6: 5.7012, 7: 6.7012, 8: 6.4966}
chance_dict:  {1: 0.1, 2: 0.25, 3: 0.37, 4: 0.5, 5: 0.62, 6: 0.75, 7: 0.87, 8: 1}


In [17]:
## Use Raw Cost

all_routes_4star_raw = Find_Opt_Route(n_list,use_actual_cost=False)
all_routes_4star_raw.sort(key=lambda x: x.cum_avg)

for node in all_routes_4star_raw[:20]:
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(node.history)))

Avg = 4.9947 ; Route = [2, 1, '+']
Avg = 5.0977 ; Route = [2, 2, 1, '+']
Avg = 5.2302 ; Route = [1, '+']
Avg = 5.2344 ; Route = [3, 1, '+']
Avg = 5.2807 ; Route = [2, 2, 2, 1, '+']
Avg = 5.4006 ; Route = [3, 2, 1, '+']
Avg = 5.4320 ; Route = [2, 2, 2, 2, 1, '+']
Avg = 5.5242 ; Route = [2, 2, 2, 2, 2, 1, '+']
Avg = 5.5684 ; Route = [2, 2, 2, 2, 2, 2, 1, '+']
Avg = 5.5772 ; Route = [3, 2, 2, 1, '+']
Avg = 5.5850 ; Route = [2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 5.5897 ; Route = [4, 1, '+']
Avg = 5.5897 ; Route = [2, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 5.5907 ; Route = [2, 2, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 5.5908 ; Route = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 5.5908 ; Route = [2, '+']
Avg = 5.7004 ; Route = [3, 2, 2, 2, 1, '+']
Avg = 5.7306 ; Route = [3, 3, 1, '+']
Avg = 5.7668 ; Route = [3, 2, 2, 2, 2, 1, '+']
Avg = 5.7718 ; Route = [4, 2, 1, '+']


In [18]:
## Use actual cost

all_routes_4star_act = Find_Opt_Route(n_list, use_actual_cost=True)
all_routes_4star_act.sort(key=lambda x: x.cum_avg)

for node in all_routes_4star_act[:20]:
    
    templist = [count_to_star_dict.get(item) if count_to_star_dict.get(item) != None else item for item in node.history]
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(templist)))

Avg = 4.9947 ; Route = ['1★', '0★', '+']
Avg = 5.0977 ; Route = ['1★', '1★', '0★', '+']
Avg = 5.2302 ; Route = ['0★', '+']
Avg = 5.2344 ; Route = ['1★+0★', '0★', '+']
Avg = 5.2807 ; Route = ['1★', '1★', '1★', '0★', '+']
Avg = 5.2909 ; Route = ['2★', '0★', '+']
Avg = 5.4006 ; Route = ['1★+0★', '1★', '0★', '+']
Avg = 5.4320 ; Route = ['1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.4730 ; Route = ['2★', '1★', '0★', '+']
Avg = 5.5242 ; Route = ['1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.5684 ; Route = ['1★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.5772 ; Route = ['1★+0★', '1★', '1★', '0★', '+']
Avg = 5.5850 ; Route = ['1★', '1★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.5897 ; Route = ['1★', '1★', '1★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.5907 ; Route = ['1★', '1★', '1★', '1★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.5908 ; Route = ['1★', '1★', '1★', '1★', '1★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 5.5908 ; Route = ['1★', '+']
Avg = 5.6236

# 4 STAR TO 5 STAR


In [19]:
## 4star to 5star

count_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
ActualCost_list = [1,2,3,3.7012,4.7012,5.7012,6.7012,6.4966,7.4966,8.4966,9.4966,10.1978,11.1978,12.1978,13.1978,11.4913]
success_chance_list = [0.01,0.1,0.17,0.25,0.31,0.37,0.43,0.5,0.56,0.62,0.68,0.75,0.81,0.87,0.93,1]

chance_dict = {}
cost_dict = {}

for count,chance in zip(count_list,success_chance_list):
    chance_dict[count] = chance
    
for count,cost in zip(count_list,ActualCost_list):
    cost_dict[count] = cost
    
n_list = [n for n in reversed(range(1,len(count_list)))]

print("n_list: ", n_list)
print("cost_dict: ", cost_dict)
print("chance_dict: ",chance_dict)

n_list:  [15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
cost_dict:  {1: 1, 2: 2, 3: 3, 4: 3.7012, 5: 4.7012, 6: 5.7012, 7: 6.7012, 8: 6.4966, 9: 7.4966, 10: 8.4966, 11: 9.4966, 12: 10.1978, 13: 11.1978, 14: 12.1978, 15: 13.1978, 16: 11.4913}
chance_dict:  {1: 0.01, 2: 0.1, 3: 0.17, 4: 0.25, 5: 0.31, 6: 0.37, 7: 0.43, 8: 0.5, 9: 0.56, 10: 0.62, 11: 0.68, 12: 0.75, 13: 0.81, 14: 0.87, 15: 0.93, 16: 1}


In [20]:
## Use Raw cost

all_routes_5star_raw = Find_Opt_Route(n_list)
all_routes_5star_raw.sort(key=lambda x: x.cum_avg)

for node in all_routes_5star_raw[:20]:
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(node.history)))
    

Avg = 9.7238 ; Route = [4, 2, 2, 2, 2, 1, '+']
Avg = 9.7531 ; Route = [4, 2, 2, 2, 1, '+']
Avg = 9.7655 ; Route = [4, 2, 2, 2, 2, 2, 1, '+']
Avg = 9.8249 ; Route = [4, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 9.8438 ; Route = [4, 3, 2, 2, 1, '+']
Avg = 9.8643 ; Route = [4, 4, 2, 1, '+']
Avg = 9.8717 ; Route = [4, 3, 2, 2, 2, 1, '+']
Avg = 9.8792 ; Route = [4, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 9.8966 ; Route = [4, 4, 2, 2, 1, '+']
Avg = 9.9067 ; Route = [5, 2, 2, 2, 1, '+']
Avg = 9.9209 ; Route = [4, 2, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 9.9228 ; Route = [4, 3, 2, 1, '+']
Avg = 9.9326 ; Route = [4, 3, 2, 2, 2, 2, 1, '+']
Avg = 9.9341 ; Route = [5, 2, 2, 2, 2, 1, '+']
Avg = 9.9495 ; Route = [4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 9.9559 ; Route = [4, 4, 1, '+']
Avg = 9.9672 ; Route = [4, 4, 2, 2, 2, 1, '+']
Avg = 9.9676 ; Route = [4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, '+']
Avg = 9.9690 ; Route = [4, 2, 2, 1, '+']
Avg = 9.9783 ; Route = [4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, '+']


In [21]:
## Use Actual Cost

all_routes_5star_act = Find_Opt_Route(n_list, use_actual_cost=True)
all_routes_5star_act.sort(key=lambda x: x.cum_avg)

for node in all_routes_5star_act[:20]:
    
    templist = [count_to_star_dict.get(item) if count_to_star_dict.get(item) != None else item for item in node.history]
    print("Avg = " + str("{:.4f}".format(node.cum_avg)) + " ; Route = " + str(Dump_Down_History(templist)))

Avg = 9.3414 ; Route = ['2★', '2★', '1★', '0★', '+']
Avg = 9.3466 ; Route = ['3★', '1★', '0★', '+']
Avg = 9.3737 ; Route = ['2★', '2★', '1★', '1★', '0★', '+']
Avg = 9.3788 ; Route = ['3★', '1★', '1★', '0★', '+']
Avg = 9.4250 ; Route = ['2★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 9.4330 ; Route = ['2★', '2★', '0★', '+']
Avg = 9.4378 ; Route = ['3★', '0★', '+']
Avg = 9.4443 ; Route = ['2★', '2★', '1★', '1★', '1★', '0★', '+']
Avg = 9.4491 ; Route = ['3★', '1★', '1★', '1★', '0★', '+']
Avg = 9.4543 ; Route = ['2★', '1★', '1★', '1★', '0★', '+']
Avg = 9.4667 ; Route = ['2★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 9.5149 ; Route = ['2★', '2★', '1★+0★', '0★', '+']
Avg = 9.5153 ; Route = ['2★', '2★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 9.5193 ; Route = ['3★', '1★+0★', '0★', '+']
Avg = 9.5197 ; Route = ['3★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 9.5229 ; Route = ['2★', '2★', '2★', '0★', '+']
Avg = 9.5261 ; Route = ['2★', '1★', '1★', '1★', '1★', '1★', '1★', '0★', '+']
Avg = 9.5

In [46]:
## Using the "Raw Cost"

print("Actual Cost of 1star gear = ", 2 )
print("Actual Cost of 2star gear = ", 2 + 1.7012)
print("Actual Cost of 3star gear = ", 2 + 1.7012 + 2.7954)
print("Actual Cost of 4star gear = ", 2 + 1.7012 + 2.7954 + 4.9947)
print("Actual Cost of 5star gear = ", 2 + 1.7012 + 2.7954 + 4.9947 + 9.7238)

Actual Cost of 1star gear =  2
Actual Cost of 2star gear =  3.7012
Actual Cost of 3star gear =  6.4966
Actual Cost of 4star gear =  11.491299999999999
Actual Cost of 5star gear =  21.2151


In [47]:
## Using the "Actual Cost"

print("Actual Cost of 1star gear = ", 2 )
print("Actual Cost of 2star gear = ", 2 + 1.7012)
print("Actual Cost of 3star gear = ", 2 + 1.7012 + 2.7954)
print("Actual Cost of 4star gear = ", 2 + 1.7012 + 2.7954 + 4.9947)
print("Actual Cost of 5star gear = ", 2 + 1.7012 + 2.7954 + 4.9947 + 9.3414)

Actual Cost of 1star gear =  2
Actual Cost of 2star gear =  3.7012
Actual Cost of 3star gear =  6.4966
Actual Cost of 4star gear =  11.491299999999999
Actual Cost of 5star gear =  20.8327


# Deeper look into one route

For people who knows Python and want to look deeper into a single route. Note that for infinite route, while the real calculation of its average cost in decision-tree go down the route for 1000 time, the individual result from each node was only recorded for the first 30 times due to memory issue. 

All routes are saved in ascending order of its average cost in their respective list variable below:

    1★ to 2★ (Raw Cost) = all_routes_2star_raw

    1★ to 2★ (Actual Cost) = all_routes_2star_act
    
    2★ to 3★ (Raw Cost) = all_routes_3star_raw

    2★ to 3★ (Actual Cost) = all_routes_3star_act
    
    3★ to 4★ (Raw Cost) = all_routes_4star_raw

    3★ to 4★ (Actual Cost) = all_routes_4star_act
    
    4★ to 5★ (Raw Cost) = all_routes_5star_raw

    4★ to 5★ (Actual Cost) = all_routes_5star_act
    
In python, you get the specific item from List by its index (first item, which is also the most optimized route, is indexed as 0, then follow by the 2nd most optimized route as index 1... and so on).

Indexing is done by square bracket right behind the list, e.g: to get most optimized route for 4★ to 5★ (Raw Cost):

    all_routes_5star_raw[0]

In [24]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [25]:
###  Input parameter   ###

lookup_route = all_routes_5star_raw[0] # Which route to lookup
raw = True # Is this route calculated with "Raw Cost", True or False


###  Analysis  ###

df = pd.DataFrame([lookup_route.history, lookup_route.cum_prob]).T
df.columns = ['Fodder Used','Prob_Node']
df["Cum_Prob"] = df['Prob_Node'].cumsum()
df["Total Used"] = df['Fodder Used'].cumsum().astype(int)

if not raw: 
    df["Fodder Used"] = df["Fodder Used"].replace(count_to_star_dict)
else:
    df['Fodder Used'] = df['Fodder Used'].astype(int)
    
df

,Fodder Used,Prob_Node,Cum_Prob,Total Used
0,4,0.25000,0.25000,4
1,2,0.13500,0.38500,6
2,2,0.12915,0.51415,8
3,2,0.11660,0.63075,10
4,2,0.09970,0.73045,12
5,1,0.05661,0.78706,13
6,1,0.04472,0.83177,14
7,1,0.03533,0.86710,15
8,1,0.02791,0.89501,16
9,1,0.02205,0.91706,17


# Whole script ends here, beyond here are just some rough work during my script development process

In [26]:
## manual validation with 5star route

# 2star 2star 1star then spam 0star ; 

#finite_avg = (0.25*4)+(0.2475*8)+(0.13065*10) ## fodder make by 100% route
finite_avg = (0.25*3.7012)+(0.2475*(3.7012*2))+(0.13065*((3.7012*2)+2)) ## fodder make by optimized route

finite_avg

3.98581756

In [27]:
## brute force

infinite_avg = 0
#total_used = 10 ## fodder make by 100% route
total_used = (3.7012*2 + 2)  ## fodder make by optimized route

for n in range(1000):
    
    total_used += 1
    cur_p = 0.75*0.67*0.74*(0.8**n)*0.2
    #       ( 1-cum_prob )
    
    cur_avg = total_used*cur_p
    infinite_avg += cur_avg
    
infinite_avg

5.35553244

In [28]:
finite_avg + infinite_avg

## Result matched with old guide

9.34135

In [40]:
## old guide's calculus? Incremental cost of taking branch = 1/P - 1 , Total cost = N + 1/P - 1

total_used = (3.7012*2 + 2)
cost = total_used + (1/0.2) - 1
prob = 1 - (0.25 + 0.2475 + 0.13065)

infinite_avg_calculus = cost * prob
infinite_avg_calculus
## result do not match with brute force method and do not match with old guide final results

4.98368244

In [41]:
## By my intuition, Incremental cost of taking branch = 1/P , Total cost = N + 1/P

total_used = (3.7012*2 + 2)
cost = total_used + (1/0.2)
prob = 1 - (0.25 + 0.2475 + 0.13065)

infinite_avg_calculus = cost * prob
infinite_avg_calculus

## result now match with brute force method and also match with old guide final results

5.35553244

In [29]:
test_node = calc_new_node(zero_node,4,USE_ACTUAL_COST=True)
test_node

Node(history=[4], fail_bonus=0.08, cum_prob=[0.25], cum_avg=0.9253, ended=False)

In [30]:
test_node = calc_new_node(test_node,4,USE_ACTUAL_COST=True)
test_node

Node(history=[4, 4], fail_bonus=0.16, cum_prob=[0.25, 0.2475], cum_avg=2.757394, ended=False)

In [31]:
test_node = calc_new_node(test_node,2,True)
test_node

Node(history=[4, 4, 2], fail_bonus=0.19, cum_prob=[0.25, 0.2475, 0.13065], cum_avg=3.98581756, ended=False)

In [32]:
test_node = calc_new_node(test_node,1,True)
test_node

Node(history=[4, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], fail_bonus=0.19, cum_prob=[0.25, 0.2475, 0.13065, 0.07437, 0.05949600000000001, 0.047596800000000015, 0.03807744000000002, 0.03046195200000001, 0.024369561600000006, 0.01949564928000001, 0.015596519424000005, 0.012477215539200007, 0.009981772431360005, 0.007985417945088006, 0.006388334356070404, 0.005110667484856324, 0.0040885339878850594, 0.0032708271903080476, 0.002616661752246438, 0.0020933294017971503, 0.0016746635214377208, 0.0013397308171501766, 0.0010717846537201413, 0.0008574277229761132, 0.0006859421783808905, 0.0005487537427047124, 0.00043900299416376994, 0.0003512023953310161, 0.0002809619162648128, 0.00022476953301185025, 0.00017981562640948022, 0.00014385250112758418, 0.00011508200090206737, 9.206560072165389e-05], cum_avg=9.34135, ended=True)

In [42]:
test_node = calc_new_node(zero_node,4,USE_ACTUAL_COST=False)
test_node

Node(history=[4], fail_bonus=0.08, cum_prob=[0.25], cum_avg=1.0, ended=False)

In [43]:
test_node = calc_new_node(test_node,4,USE_ACTUAL_COST=True)
test_node

Node(history=[4, 4], fail_bonus=0.16, cum_prob=[0.25, 0.2475], cum_avg=2.832094, ended=False)

In [44]:
test_node = calc_new_node(test_node,2,True)
test_node

Node(history=[4, 4, 2], fail_bonus=0.19, cum_prob=[0.25, 0.2475, 0.13065], cum_avg=4.06051756, ended=False)

In [45]:
test_node = calc_new_node(test_node,1,True)
test_node

Node(history=[4, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], fail_bonus=0.19, cum_prob=[0.25, 0.2475, 0.13065, 0.07437, 0.05949600000000001, 0.047596800000000015, 0.03807744000000002, 0.03046195200000001, 0.024369561600000006, 0.01949564928000001, 0.015596519424000005, 0.012477215539200007, 0.009981772431360005, 0.007985417945088006, 0.006388334356070404, 0.005110667484856324, 0.0040885339878850594, 0.0032708271903080476, 0.002616661752246438, 0.0020933294017971503, 0.0016746635214377208, 0.0013397308171501766, 0.0010717846537201413, 0.0008574277229761132, 0.0006859421783808905, 0.0005487537427047124, 0.00043900299416376994, 0.0003512023953310161, 0.0002809619162648128, 0.00022476953301185025, 0.00017981562640948022, 0.00014385250112758418, 0.00011508200090206737, 9.206560072165389e-05], cum_avg=9.41605, ended=True)

# Old System 4 star to 5 star

In [152]:
## 4star to 5star (old system)

count_list = [1,2,4,8,16]
ActualCost_list = [1,2,3.7012,6.4966,11.4913] 
success_chance_list = [0.01,0.1,0.25,0.5,1]

chance_dict = {}
cost_dict = {}

for count,chance in zip(count_list,success_chance_list):
    chance_dict[count] = chance
    
for count,cost in zip(count_list,ActualCost_list):
    cost_dict[count] = cost
    
n_list = [8,4,2,1]

print("n_list: ", n_list)
print("cost_dict: ", cost_dict)
print("chance_dict: ",chance_dict)

n_list:  [8, 4, 2, 1]
cost_dict:  {1: 1, 2: 2, 4: 3.7012, 8: 6.4966, 16: 11.4913}
chance_dict:  {1: 0.01, 2: 0.1, 4: 0.25, 8: 0.5, 16: 1}


In [ ]:
ended_nodes = []
use_actual_cost = False

for n in n_list: ## Different starting point

    # Initiate the first node into the list
    nodes_list = [calc_new_node(zero_node,n,use_actual_cost)]

    while len(nodes_list) >= 1: # As long as there is still nodes not ended

        cur_node = nodes_list[-1] # Always process the last node
        del nodes_list[-1] # Remove this node from list since it had been processed

        ## A very special case for spamming only 1 gear at 5star, node ended the moment it is created because it enter infinite case immediately
        if cur_node.ended: continue

        n2_list = n_list[n_list.index(cur_node.history[-1]):]
        n2_list.reverse() # List of possible route to take (number of gear to use for starring for each route)

        for n2 in n2_list: # Calc the next node for each route

            temp_node = calc_new_node(cur_node,n2,use_actual_cost)

            if temp_node.ended: # This node calculation is done, result can be reported !
                ended_nodes.append(temp_node)

            else: # Incomplete node, add to nodes_list for further processing
                nodes_list.append(temp_node)

ended_nodes.sort(key=lambda x: x.cum_avg)
for node in ended_nodes:
    print("Avg = " + str(node.cum_avg) + " ; Route = " + str(Dump_Down_History(node.history)))

In [ ]:
ended_nodes = []
use_actual_cost = True

for n in n_list: ## Different starting point

    # Initiate the first node into the list
    nodes_list = [calc_new_node(zero_node,n,use_actual_cost)]

    while len(nodes_list) >= 1: # As long as there is still nodes not ended

        cur_node = nodes_list[-1] # Always process the last node
        del nodes_list[-1] # Remove this node from list since it had been processed

        ## A very special case for spamming only 1 gear at 5star, node ended the moment it is created because it enter infinite case immediately
        if cur_node.ended: continue

        n2_list = n_list[n_list.index(cur_node.history[-1]):]
        n2_list.reverse() # List of possible route to take (number of gear to use for starring for each route)

        for n2 in n2_list: # Calc the next node for each route

            temp_node = calc_new_node(cur_node,n2,use_actual_cost)

            if temp_node.ended: # This node calculation is done, result can be reported !
                ended_nodes.append(temp_node)

            else: # Incomplete node, add to nodes_list for further processing
                nodes_list.append(temp_node)

ended_nodes.sort(key=lambda x: x.cum_avg)
for node in all_routes:
    
    templist = [count_to_star_dict.get(item) if count_to_star_dict.get(item) != None else item for item in node.history]
    print("Avg = " + str(node.cum_avg) + " ; Route = " + str(Dump_Down_History(templist)))

In [105]:
n_list[n_list.index(8):]

[8, 4, 2, 1]

In [92]:
for node in ended_nodes:
    print(node.cum_avg,node.history)

1.701212 [1, 1, 1, 1, 1]


In [81]:
len(ended_nodes)

11

In [71]:
node1 = calc_new_node(zero_node,3)
node1

Node(history=[3], fail_bonus=0.25, cum_prob=0.75, cum_avg=2.25, ended=False)

In [74]:
node2 = calc_new_node(node1,3)
node2

Node(history=[3, 3], fail_bonus=0.5, cum_prob=1.0, cum_avg=3.75, ended=True)

In [82]:
print(node2)

Node(history=[3, 3], fail_bonus=0.5, cum_prob=1.0, cum_avg=3.75, ended=True)


In [16]:
min(1.1,1)

1

In [19]:
cur_chance = 1.1
x = cur_chance >= 1

x

True

In [51]:
def Round_Down(float_number):
    return (int(float_number*100))/100

In [58]:
Round_Down(0.1/3)

0.03

In [78]:
infinite_avg = 0
total_used = 0  ## fodder make by cheat route

for n in range(1000):
    
    total_used += 1
    cur_p = (0.99**n)*0.01
    
    cur_avg = total_used*cur_p
    infinite_avg += cur_avg
    
infinite_avg

99.95251162784835

In [50]:
1/0.01

100.0

In [61]:
6.4966+3.7012

10.1978

In [72]:
d = {1:1,2:2,3:3,4:3.7}
l = [5,4,4,2,2,1]

In [118]:
[d.get(item) if d.get(item)!=None else item for item in l]

[5, 3.7, 3.7, 2, 2, 1]

In [68]:
d.get(4)

3.7

In [80]:
Dump_Down_History([5,4,3,2,1,1])

[5, 4, 3, 2, 1, '+']

In [63]:
l = [1,2,3,4,5]
l[:-1]

[1, 2, 3, 4]